<a href="https://colab.research.google.com/github/SUREN37/Prompt-Engineering/blob/Prompt-Engineering/Zero_shot_Prompting_with_Local_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install -q llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 MB 17.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 6.3 MB/s eta 0:00:00


In [3]:
!pip show llama_cpp_python

Name: llama_cpp_python
Version: 0.2.79
Summary: Python bindings for the llama.cpp library
Home-page: 
Author: 
Author-email: Andrei Betlen <abetlen@gmail.com>
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: diskcache, jinja2, numpy, typing-extensions
Required-by: 


In [8]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

In [6]:
model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
model_basename = "mistral-7b-instruct-v0.2.Q5_K_M.gguf"
model_path = hf_hub_download(
    repo_id = model_name,
    filename = model_basename
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


mistral-7b-instruct-v0.2.Q5_K_M.gguf:   0%|          | 0.00/5.13G [00:00<?, ?B/s]

In [9]:
lcpp_llm = Llama(
    model_path = model_path,
    n_threads = 2,
    n_batch = 512,
    n_gpu_layers = 43,
    n_ctx = 4096
)

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /root/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.2-GGUF/snapshots/3a6fbf4a41a1d52e415a4958cde6856d34b2db93/mistral-7b-instruct-v0.2.Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loa

In [10]:
system_message = """ Your are an Text summarizer
summarize from the input text given"""

In [11]:
user_message = """ Investing in stocks can be a lucrative way to grow your wealth, but there are also risks involved. Some pros of investing in stocks include the potential for high returns on investment and the ability to diversify your portfolio. However, some cons include market volatility, inflation risk, and the possibility of losing money if the company's performance declines or the overall market experiences a downturn"""

In [12]:
prompt_template = """<s>[INST]\n <<SYS>> \n {system_message} \n <<SYS>>'''{user_message}''' /n[/INST]"""

In [13]:
def generate_prompt(system_message,user_input):
  try:
    prompt = prompt_template.format(system_message = system_message,user_message = user_message)
    return prompt
  except Exception as e:
    print(e)

In [19]:
prompt = generate_prompt(system_message,user_message)

In [16]:
print(prompt)

<s>[INST]
 <<SYS>> 
  Your are an Text summarizer
summarize and answer the questions from the input text given 
 <<SYS>>''' Investing in stocks can be a lucrative way to grow your wealth, but there are also risks involved. Some pros of investing in stocks include the potential for high returns on investment and the ability to diversify your portfolio. However, some cons include market volatility, inflation risk, and the possibility of losing money if the company's performance declines or the overall market experiences a downturn''' /n[/INST]


In [22]:
response = lcpp_llm(
    prompt = prompt,
    max_tokens = 512,
    temperature = 0,
    top_p = 0.95,
    repeat_penalty = 1.2,
    top_k = 50,
    echo = False
)

/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1054: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(

llama_print_timings:        load time =     505.51 ms
llama_print_timings:      sample time =      73.59 ms /   113 runs   (    0.65 ms per token,  1535.62 tokens per second)
llama_print_timings: prompt eval time =     504.73 ms /   125 tokens (    4.04 ms per token,   247.66 tokens per second)
llama_print_timings:        eval time =    3265.23 ms /   112 runs   (   29.15 ms per token,    34.30 tokens per second)
llama_print_timings:       total time =    3923.50 ms /   237 tokens


In [23]:
response

{'id': 'cmpl-0ff53750-858f-4640-b325-a3d5cab51788',
 'object': 'text_completion',
 'created': 1719163977,
 'model': '/root/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.2-GGUF/snapshots/3a6fbf4a41a1d52e415a4958cde6856d34b2db93/mistral-7b-instruct-v0.2.Q5_K_M.gguf',
 'choices': [{'text': " The text discusses both advantages and disadvantages of investing in stocks for wealth growth. Here are the key points:\n\nPros:\n1. High potential returns on investment\n2. Diversification opportunity for your portfolio\n\nCons:\n1. Market volatility, which can lead to unpredictable price fluctuations\n2. Inflation risk, as stock values may not keep pace with inflation over long periods\n3. Risk of losing money if a company's performance declines or the market experiences downturns.",
   'index': 0,
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 125,
  'completion_tokens': 112,
  'total_tokens': 237}}